### Before running the code make sure that you have baseline model saved in Folder ./Models

Then add its name down here:


In [ ]:
model_name = ""  # "model_baseline_EPOCH10" 

if model_name == "":
    print("Please specify model name")
    exit(1)

In [81]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
import re

In [82]:
test_set = input("Enter the test name : \n")

In [84]:
# Define Diacritics
KASRA = "\u0650"
DAMMA = "\u064F"
FATHA = "\u064E"
KASRATAN = "\u064D"
DAMMATAN = "\u064C"
FATHATAN = "\u064B"
SUKUN = "\u0652"
SHADDA = "\u0651"
DAMMA_SHADDA = DAMMA + SHADDA
SHADDA_DAMMA = SHADDA + DAMMA
FATHA_SHADDA = FATHA + SHADDA
SHADDA_FATHA = SHADDA + FATHA
KASRA_SHADDA = KASRA + SHADDA
SHADDA_KASRA = SHADDA + KASRA
DAMMATAN_SHADDA = DAMMATAN + SHADDA
SHADDA_DAMMATAN = SHADDA + DAMMATAN
FATHATAN_SHADDA = FATHATAN + SHADDA
SHADDA_FATHATAN = SHADDA + FATHATAN
KASRATAN_SHADDA = KASRATAN + SHADDA
SHADDA_KASRATAN = SHADDA + KASRATAN
EMPTY = "_"
PAD_LABLE = "pad"
DIACRITICS = [
    KASRA,
    DAMMA,
    FATHA,
    KASRATAN,
    DAMMATAN,
    FATHATAN,
    SUKUN,
    SHADDA,
    DAMMA_SHADDA,
    SHADDA_DAMMA,
    FATHA_SHADDA,
    SHADDA_FATHA,
    KASRA_SHADDA,
    SHADDA_KASRA,
    DAMMATAN_SHADDA,
    SHADDA_DAMMATAN,
    FATHATAN_SHADDA,
    SHADDA_FATHATAN,
    KASRATAN_SHADDA,
    SHADDA_KASRATAN,
    EMPTY,
    PAD_LABLE,
]

DIACRITICS_MAP = [
    FATHA,
    FATHATAN,
    DAMMA,
    DAMMATAN,
    KASRA,
    KASRATAN,
    SUKUN,
    SHADDA,
    SHADDA_FATHA,
    SHADDA_FATHATAN,
    SHADDA_DAMMA,
    SHADDA_DAMMATAN,
    SHADDA_KASRA,
    SHADDA_KASRATAN,
    EMPTY,
    PAD_LABLE,
]

In [85]:
# This function is responsible for mapping diacritics to their corresponding strings
def diacritic_to_str(diacritic):
    if diacritic == SHADDA:
        diacritic = "SHADDA"
    elif diacritic == KASRA:
        diacritic = "KASRA"
    elif diacritic == DAMMA:
        diacritic = "DAMMA"
    elif diacritic == FATHA:
        diacritic = "FATHA"
    elif diacritic == KASRATAN:
        diacritic = "KASRATAN"
    elif diacritic == DAMMATAN:
        diacritic = "DAMMATAN"
    elif diacritic == FATHATAN:
        diacritic = "FATHATAN"
    elif diacritic == SUKUN:
        diacritic = "SUKUN"
    elif diacritic == DAMMA_SHADDA or diacritic == SHADDA_DAMMA:
        diacritic = "SHADDA_DAMMA"
    elif diacritic == FATHA_SHADDA or diacritic == SHADDA_FATHA:
        diacritic = "SHADDA_FATHA"
    elif diacritic == KASRA_SHADDA or diacritic == SHADDA_KASRA:
        diacritic = "SHADDA_KASRA"
    elif diacritic == DAMMATAN_SHADDA or diacritic == SHADDA_DAMMATAN:
        diacritic = "SHADDA_DAMMATAN"
    elif diacritic == FATHATAN_SHADDA or diacritic == SHADDA_FATHATAN:
        diacritic = "SHADDA_FATHATAN"
    elif diacritic == KASRATAN_SHADDA or diacritic == SHADDA_KASRATAN:
        diacritic = "SHADDA_KASRATAN"
    elif diacritic=="pad":
      diacritic = "pad"
    else:  # EMPTY
        diacritic = "_"
    return diacritic

In [86]:
# filter data takes a list of strings and removes unwanted patterns
def filter_data(data: str) -> str:
    # data = re.sub(r"\( \d+ (/ \d+)? \)", "", data)
    # remove all numbers
    data = re.sub(r"\d+", "", data)
    # regex to remove all special characters
    data = re.sub(r"[][//,;\?؟()$:\-{}_*؛:«»`–\"~!']", "", data)
    # remove all english letters
    data = re.sub(r"[a-zA-Z]", "", data)
    # Substituting multiple spaces with single space
    data = re.sub(r"([^\S\n])+", " ", data, flags=re.I)
    return data

In [87]:
def get_data_chars(data_set):
    original_labels = ["" for _ in range(len(data_set))]
    sentences = ["" for _ in range(len(data_set))]
    for i in range(len(data_set)):
        line = data_set[i]
        line_without_diacritics = []
        line_labels = []
        for word in line.split():
            if word == "":
                continue
            j = 0
            while j < len(word):
                if (
                    j <= len(word) - 2
                    and word[j] in DIACRITICS
                    and word[j + 1] in DIACRITICS
                ):
                    line_labels.pop()
                    line_labels.append(
                        diacritic_to_str(word[j] + word[j + 1])
                    )  # lable of the word
                    j += 1
                else:
                    if word[j] in DIACRITICS:
                        # pop
                        line_labels.pop()
                        line_labels.append(diacritic_to_str(word[j]))  # lable of the word
                    else:
                        line_labels.append(diacritic_to_str(word[j]))  # lable of the word
                        line_without_diacritics.append(word[j])

                j += 1

        sentences[i] = " ".join(line_without_diacritics)
        original_labels[i] = " ".join(line_labels)
    sentences = list(filter(None, sentences))
    original_labels = list(filter(None, original_labels))

    return sentences, original_labels

In [88]:
def get_vocab(vocab_path, tags_path):
    vocab = {}
    with open(vocab_path) as f:
        for i, l in enumerate(f.read().splitlines()):
            vocab[l] = i  # to avoid the 0
        # loading tags (we require this to map tags to their indices)
    vocab['<PAD>'] = len(vocab) # 35180
    tag_map = {}
    with open(tags_path) as f:
        for i, t in enumerate(f.read().splitlines()):
            tag_map[t] = i

    return vocab, tag_map

def get_params(vocab, tag_map, sentences_file, labels_file):
    sentences = []
    labels = []

    for sentence in sentences_file:
        # replace each token by its index if it is in vocab
        # else use index of UNK_WORD
        s = [vocab[token] if token in vocab
                else vocab['UNK']
                for token in sentence.split(' ')]
        sentences.append(s)

    for sentence in labels_file:
        # replace each label by its index
        s = sentence.split(' ')
        # remove empty strings
        s = list(filter(None, s))
        l = [tag_map[label] for label in s] # I added plus 1 here
        labels.append(l)
    return sentences, labels, len(sentences)


In [89]:
# filter the data
test_set = filter_data(test_set)
# get space indecies
space_indecies = [m.start() for m in re.finditer(" ", test_set)]
# split the data into lines
test_set = re.split(r"[.،]", test_set)
# remove empty lines
test_set = list(filter(None, test_set))

In [90]:
test_chars, test_labels_chars = get_data_chars(test_set)

# Constants


In [91]:
MODEL = "BI_LSTM"
NUM_LAYERS = 2
EMBEDDING_SIZE = 300

# Importing and discovering the data


In [92]:
vocab, tag_map = get_vocab('./Dataset/unique_chars.txt', './Dataset/unique_labels.txt')
test_sentences1, test_labels1, test_size1 = get_params(vocab, tag_map, test_chars, test_labels_chars)

# DiacritiserDataset

The class that impelements the dataset for Diacritization


In [93]:
class DiacritiserDataset(torch.utils.data.Dataset):

  def __init__(self, x, y, pad):
    self.x = nn.utils.rnn.pad_sequence([torch.tensor(i) for i in x], padding_value=pad,batch_first = True)
    self.y = nn.utils.rnn.pad_sequence([torch.tensor(i) for i in y], padding_value=tag_map["pad"],batch_first = True)
    print('The max length of the sentences is', self.x.shape[1])
    print('The max length of the labels is', self.y.shape[1])
  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

# Classifiers

The class that implementss the pytorch model for arabic diacritic classification


In [94]:
class ArabicDiacriticsClassifier(nn.Module):
  def __init__(self, vocab_size = 128137, num_layers = 3, embedding_dim = 512, hidden_size=256, n_classes=len(tag_map)):
    super(ArabicDiacriticsClassifier, self).__init__()
    # (1) Create the embedding layer
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    # import gensim.downloader as api
    # self.embedding.weight.data.copy_(torch.from_numpy(api.load('word2vec-google-news-300').vectors[:1000])) WORD2VEc
    # import fasttext.util
    # self.embedding.weight.data.copy_(torch.from_numpy(fasttext.util.download_model('en', if_exists='ignore').get_input_matrix()[:1000]))

    # (2) Create an LSTM layer with hidden size = hidden_size and batch_first = True
    self.num_layers = num_layers
    self.hidden_size = hidden_size


    self.lstm = nn.LSTM(embedding_dim, hidden_size,num_layers, batch_first=True, bidirectional=True)
    self.linear = nn.Linear(2 * hidden_size, n_classes)

    # (3) Create a linear layer with number of neorons = n_classes
    # self.linear = nn.Linear(hidden_size, n_classes)


  def forward(self, sentences):
    embeddings = self.embedding(sentences)

    # BIDIRECTIONAL
    # Initialize hidden states for bidirectional LSTM
    # h0 = torch.zeros(self.num_layers*2, embeddings.size(0), self.hidden_size).to(sentences.device)
    # c0 = torch.zeros(self.num_layers*2, embeddings.size(0), self.hidden_size).to(sentences.device)
    # lstm_out, (a, b) = self.lstm(embeddings, (h0, c0))

    # LSTM
    lstm1_out, (a,b) = self.lstm(embeddings)
    final_output = self.linear(lstm1_out)
    # final_output = self.linear(lstm_out[:, -1, :])
    return final_output

In [95]:
model = ArabicDiacriticsClassifier()
print(model)

ArabicDiacriticsClassifier(
  (embedding): Embedding(128137, 512)
  (lstm): LSTM(512, 256, num_layers=3, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=16, bias=True)
)


In [96]:
test_dataset = DiacritiserDataset(test_sentences1, test_labels1, vocab['<PAD>'])

The max length of the sentences is 20
The max length of the labels is 20


In [97]:
def load_model(model, model_name):
  model.load_state_dict(torch.load(f'./Models/{model_name}'))
  return model

In [98]:
model = load_model(model, model_name)

# Evaluation


In [99]:
diacritic_results = []
gold_results = []
test_input_list = []

def evaluate(model, test_dataset, batch_size=64):
  # (1) create the test data loader
  test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

  # GPU Configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()

  # (2) disable gradients
  with torch.no_grad():

    for test_input, test_label in tqdm(test_dataloader):
      # (3) move the test input to the device
      test_label = test_label.to(device)

      # (4) move the test label to the device
      test_input = test_input.to(device)

      # (5) do the forward pass
      output = model(test_input)
      prediction = output.argmax(2)


      diacritic_results.extend(np.array(prediction.cpu().data).flatten())
      gold_results.extend(np.array(test_label.cpu().data).flatten())
      test_input_list.extend(np.array(test_input.cpu().data).flatten())

### Evaluatio and Save


In [100]:
evaluate(model, test_dataset)

100%|██████████| 1/1 [00:00<00:00, 58.39it/s]


In [101]:
def DER():
  der = 0
  total_size = 0
  for i in range(len(diacritic_results)):
    if test_input_list[i] != vocab['<PAD>']: # Do not include padding in DER calculations
      if diacritic_results[i] != gold_results[i] : # Miss Classification
        der += 1
      total_size += 1
  der /= total_size
  der *= 100
  print("DER = ",der,"%")
  print("Accuracy = ",100 - der,"%")

In [102]:
filtered_diacritic_results = [] # diacrtic results without paddings
filtered_inputs = [] # inputs without paddings
LIST_OF_DIACRITICS = [
    "FATHA",
    "FATHATAN",
    "DAMMA",
    "DAMMATAN",
    "KASRA",
    "KASRATAN",
    "SUKUN",
    "SHADDA",
    "SHADDA_FATHA",
    "SHADDA_FATHATAN",
    "SHADDA_DAMMA",
    "SHADDA_DAMMATAN",
    "SHADDA_KASRA",
    "SHADDA_KASRATAN",
    "_"
]
LIST_OF_ARABIC_LETTERS = list(vocab.keys())



for i in range(len(diacritic_results)):
  if test_input_list[i] != vocab['<PAD>']:
    filtered_diacritic_results.append(diacritic_results[i])
    filtered_inputs.append(test_input_list[i])

index = len(filtered_diacritic_results)

inputs = [LIST_OF_ARABIC_LETTERS[filtered_inputs[i]] for i in range(index)]
model_prediction = [LIST_OF_DIACRITICS[filtered_diacritic_results[i]] for i in range(index)]


In [103]:
data_length = len(filtered_diacritic_results)
df = pd.DataFrame(
    {
    'ID': range(len(filtered_diacritic_results[0: data_length])),
    'label': filtered_diacritic_results[0: data_length],
    'input':  inputs,
    'diactric':  model_prediction,
    })

df.to_csv(f'./Results/result_{model_name}.csv', index=False)

In [105]:
output = ""
output_len = 0
j = 0
for i in range(len(diacritic_results)):
    if test_input_list[i] != vocab['<PAD>']:
        output += LIST_OF_ARABIC_LETTERS[test_input_list[i]]
        diacritic =  DIACRITICS_MAP[diacritic_results[i]] if DIACRITICS_MAP[diacritic_results[i]] != "_" else ""
        output += diacritic
        output_len += 2
        if j < len(space_indecies) and output_len // 2 == space_indecies[j]:
            j += 1
            output += " "
            output_len += 2 
print(output)

قَالَ الزَّرْكَشِيُّ رَضِيَ اللَّهُ عَنْهُ
